In [1]:
def DUB2DC( x, y, x1, y1, x2, y2 ): ## given global coordinates
    ## transform from global coordinates to panel coordinates (coordinates)
    xp, yp, xp1, yp1, xp2, yp2, alpha = GlobalToPanelCoords( x, y, x1, y1, x2, y2 )
    
    ## compute induced velocity by the doublet (in panel coordinates)
    up = ( yp / ( (xp-xp1)**2 + yp**2 ) - yp / ( (xp-xp2)**2+yp**2 ) )/2/pi
    vp = -( (xp-xp1) / ( (xp-xp1)**2 + yp**2 ) - (xp-xp2) / ( (xp-xp2)**2+yp**2 ) )/2/pi
    
    ## transform from panel coordinates to global coordinates (velocity)
    c = cos(alpha); s = sin(alpha)
    u = c*up - s*vp; v = s*up + c*vp
    return u, v

def NeumannConstantStrengthDoubletMethod( X, Y, N, U0, AOA ):
    
    # generate geometry
    # for N panels we need N+1 points
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        tck = createSplines( X, Y )
    x, y = computeNodeLocations( N+1, tck )
#     tx, ty = computeTangent( N+1, tck )
    binormal = computeBinormal( N+1, tck )

    ## generate collocation point data
    xc = ( x[1:] + x[:-1] )/2; yc = ( y[1:] + y[:-1] )/2;
    panelAlpha = np.arctan2( np.diff(y), np.diff(x) )
    txc = np.cos( panelAlpha ); tyc = np.sin( panelAlpha )
    nxc = np.sin( panelAlpha ); nyc = -np.cos( panelAlpha )
    
    ## generate wake geometry
    tckw = createWakeGeometry( X, Y, AOA )
    xw, yw = computeNodeLocations( N+1, tckw )
    
    # generate a (N+2)x(N+2) system of equations
    # where the last row is to apply the kutta-condition
    A = np.zeros( (N + 1, N+1) )
    b = np.zeros( N + 1 )
    
    ## row of matrix
    for i in range( N ):
        b[i] = -U0 * ( cos( AOA * pi / 180 ) * nxc[i] + sin( AOA * pi / 180 ) * nyc[i] )
        for j in range( N ):
            ## first panel point & second panel point
            u, v = DUB2DC( xc[i], yc[i], x[j], y[j], x[j+1], y[j+1] )
            A[i, j] = (u*nxc[i] + v*nyc[i])
            
            
    ## include wake effect
    for i in range( N ):
        for j in range( N ):
            u, v = DUB2DC( xc[i], yc[i], xw[j], yw[j], xw[j+1], yw[j+1] )
            A[i, -1] = A[i, -1] + (u*nxc[i] + v*nyc[i])
    
    ## include Kutta Condition
    A[N, 0] = 1; A[N, -2] = -1; A[N, -1] = 1     

    return A, b, x, y, xc, yc, txc, tyc


print("ComputeAlpha")
print( ComputeAlpha( 1, 0, 0, 1) )
print( pi/4 +pi/2 )
print("")
print( ComputeAlpha( 1, 0, 0, -1) )
print( -(pi/4 +pi/2) )
print("\n")

print("GlobalToPanelCoords")
print( GlobalToPanelCoords( 5, 3, 4, 1, 2, 2 ) )
print( "(", 0, -5**.5, 0, 0, 5**.5, 0, ")")
print("")
print( GlobalToPanelCoords( 3, -1, 4, 1, 2, 2 ) )
print( "(", 0, 5**.5, 0, 0, 5**.5, 0, ")")
print("\n")



print("DUB2DC")
u, v = DUB2DC(0, 0, 1, 0, -1, 0)
print(u, v)  
print( 0.0, 1/pi )
print("")
u, v = DUB2DC(0, 0, -1, 0, 1, 0)
print(u, v)  
print( 0.0, -1/pi )
print("")
u, v = DUB2DC(0, 0, 1/np.sqrt(2), -1/np.sqrt(2), -1/np.sqrt(2), 1/np.sqrt(2))
print(u, v)  
print( 1/pi/2**.5, 1/pi/2**.5 )







ComputeAlpha


NameError: name 'ComputeAlpha' is not defined